In [356]:
import numpy as np
from scipy.optimize import fsolve

In [357]:
# List of key parameters




mumax4=0.17 #1/day
thetamax4=0.57 #molATP/cmolX/day in Unit4
kla=20 #1/day
Ks_mu4=0.002 #cmol/L
Co_sat=7/1000/32 #mol/l
Ko_mu4=Co_sat*0.15 #mol/L
Ks_theta4=Ks_mu4/1e6 #cmol/L
Ko_theta4=Ko_mu4/1e6 #mol/L
Q1=1.5  #L

In [358]:
MM = np.array([12,1,16,14,31,23])

In [359]:
#Composition specs 'C H1.854 O0.48 N0.095 P0.0036 Na0.0016'

In [360]:
#Composition specifications:
#order         C;   H;  O;  N;  P;  Na;   

S1 = np.array([1,1.854,0.48,0.095,0.0036,0.0016])
S2 = np.array([1,1.75,0.59,0.125,0.005825,0.00172])
S3 = np.array([1,1.46,0.6,0.02,0.0025,0.0023])
S4 = X4 = np.array([1,1.7,0.55,0.13,0.02,0])
S5 = np.array([1,1.7,0.75,0.02,0.00275,0.00118])
X3 = np.array([1,1.8,0.5,0.08,0.02,0])

Calculating the amount of food Ben eats and all the NE's associated with this.

In [361]:
wetFractionFood = 0.6
nonEdibleFraction = 0.6

In [362]:
kiloJoulesRequired = 9000
#                                     Prot, Fat, Carbs
foodFractionsDryMassBasis = np.array([0.36,0.21,0.43])
energyPerDryGramFoodGroup = np.array([17,38,17])
energyPerDryGramFood = np.sum(foodFractionsDryMassBasis * energyPerDryGramFoodGroup)
massChonDryFoodPerDay = kiloJoulesRequired / energyPerDryGramFood
molsChonDryFoodPerDay = massChonDryFoodPerDay / (MM[0]*S1[0] + MM[1]*S1[1] + MM[2]*S1[2] + MM[3]*S1[3])
massDryFoodPerDay = molsChonDryFoodPerDay * np.sum(MM*S1)

massWetFoodPerDay = massDryFoodPerDay / (1 - wetFractionFood)

massWaterInFoodPerDay = massWetFoodPerDay - massDryFoodPerDay

massNonEdibleDryFoodPerDay = massDryFoodPerDay / (1 - nonEdibleFraction) * nonEdibleFraction

massNonEdibleWetFoodPerDay = massNonEdibleDryFoodPerDay / (1 - wetFractionFood)

massWaterInNonEdibleFoodPerDay = massNonEdibleWetFoodPerDay - massNonEdibleDryFoodPerDay

print('Mass dry food per day =',np.round(massDryFoodPerDay,2))
print('Mass wet food per day =',np.round(massWetFoodPerDay,2))

print('\nMass dry non-edible food per day =',np.round(massNonEdibleDryFoodPerDay,2))
print('Mass wet non-edible food per day =',np.round(massNonEdibleWetFoodPerDay,2))

print('\nMass water in edible food per day =',np.round(massWaterInFoodPerDay,2))
print('Mass water in non-edible food per day =',np.round(massWaterInNonEdibleFoodPerDay,2))

Mass dry food per day = 423.09
Mass wet food per day = 1057.73

Mass dry non-edible food per day = 634.64
Mass wet non-edible food per day = 1586.6

Mass water in edible food per day = 634.64
Mass water in non-edible food per day = 951.96


In [363]:
molsDryFoodPerDay = molsChonDryFoodPerDay
molsNonEdibleDryFoodPerDay = massNonEdibleDryFoodPerDay / np.sum(MM * S3)

print('Mols dry food per day =',np.round(molsDryFoodPerDay,3))
print('Mols dry non-edible food per day =',np.round(molsNonEdibleDryFoodPerDay,3))

Mols dry food per day = 18.385
Mols dry non-edible food per day = 27.04


Solving the Hydroponics unit:

In [364]:
#                              P    Na   NH3   H2O   CO2   O2   S1      S3
hydroponicsMatrixA=np.matrix([[0,   0,   0,    0,    1,    0,   1,       1     ],  #C
                              [0,   0,   3,    2,    0,    0,   1.853 ,  1.46  ],  #H
                              [0,   0,   0,    1,    2,    2,   0.479 ,  0.6   ],  #O
                              [0,   0,   1,    0,    0,    0,   0.096 ,  0.02  ],  #N
                              [1,   0,   0,    0,    0,    0,   0.0036,  0.0025],  #P
                              [0,   1,   0,    0,    0,    0,   0.0016,  0.0023],  #Na
                              [0,   0,   0,    0,    0,    0,   1,       0     ],  #S1 fed to ben
                              [0,   0,   0,    0,    0,    0,   0,       1     ]]) #S2 non edible food

hydroponicsMatrixB=np.matrix([0, 0, 0, 0, 0, 0, molsDryFoodPerDay, molsNonEdibleDryFoodPerDay ]).T

hydroponicsRates=np.linalg.solve(hydroponicsMatrixA, hydroponicsMatrixB)

U1RateP, U1RateNa, U1RateNH3, U1RateH2O, U1RateCO2, U1RateO2, U1RateS1, U1RateS3 = hydroponicsRates.flat

print(hydroponicsRates)

[[ -0.13378747]
 [ -0.09160862]
 [ -2.30580016]
 [-33.31457732]
 [-45.42539987]
 [ 49.56738668]
 [ 18.38542319]
 [ 27.03997669]]


Modelling the Urine:

Urine can be modelled as 40g of urea exiting the body on a daily basis. Allow for a seperate P and Na stream in your unit 2 matrix and note that this should be added to the total urine stream. Ben produces 1.5 liter of urine a day. The urea in the urine will spontaneously undergo the following decomposition:

$$CH_4ON_2 +H_2O \to CO_2+2NH_3$$

Ensure to calculate a $NH_3$ concentration in the urine.

In [365]:
massUreaPerDay = 40
molarMassUrea = 12 + 4 + 16 + 28
molsUreaPerDay = massUreaPerDay / molarMassUrea

Solving Ben's metabolism:

In [366]:
#                      P    Na    H2O   CO2   O2   S1      S2     U
benMatrixA=np.matrix([[0,   0,    0,    1,    0,   1,      S2[0], 1],  #C
                      [0,   0,    2,    0,    0,   1.853 , S2[1], 4],  #H
                      [0,   0,    1,    2,    2,   0.479 , S2[2], 1],  #O
                      [0,   0,    0,    0,    0,   0.096 , S2[3], 2],  #N
                      [1,   0,    0,    0,    0,   0.0036, S2[4], 0],  #P
                      [0,   1,    0,    0,    0,   0.0016, S2[5], 0],  #Na
                      [0,   0,    0,    0,    0,   1,      0,     0],  #S1 fed to ben
                      [0,   0,    0,    0,    0,   0,      0,     1]]) #U leaving

benMatrixB = np.matrix([0, 0, 0, 0, 0, 0, -molsDryFoodPerDay, molsUreaPerDay ]).T

benRates = np.linalg.solve(benMatrixA, benMatrixB)

U2RateP, U2RateNa, U2RateH2O, U2RateCO2, U2RateO2, U2RateS1, U2RateS2, U2RateU = benRates.flat

print(benRates)

[[  0.04607183]
 [  0.02347694]
 [ 12.6790902 ]
 [ 14.26541818]
 [-17.55372257]
 [-18.38542319]
 [  3.45333834]
 [  0.66666667]]


Anaerobic digester

In [367]:
#Start by calculating the mass of humanure:
waterFractionHumanure = 0.75
molarMassHumanure = np.sum(MM * S2)
molsDryHumanurePerDay = U2RateS2
massDryHumanurePerDay = molsDryHumanurePerDay * molarMassHumanure
print('Mass dry humanure =',massDryHumanurePerDay)

massWaterInHumanure = massDryHumanurePerDay / (1 - waterFractionHumanure) * waterFractionHumanure
print(massWaterInHumanure)

Mass dry humanure = 86.88645885282592
260.6593765584778


In [368]:
#Calculate the mass of Substrate and thus the required amount of water:
guessS4 = 0.0744599

substrate3MolsPerElement = molsNonEdibleDryFoodPerDay * S3 + U2RateS2 * S2 + guessS4 * S4
print('Substrate mols per element fed to Unit 3:',np.round(substrate3MolsPerElement,2))

U3S = substrate3MolsPerElement / substrate3MolsPerElement[0]      #This is the cmol formula for the AD substrate
print('\nAD Substrate fed to unit 3 cmol formula:',np.round(U3S,4))

molsADSubstrateUnit3 = molsNonEdibleDryFoodPerDay + U2RateS2 + guessS4
print('\nMols AD substrate into unit 3 =',molsADSubstrateUnit3)

massADSubstrateUnit3 = molsADSubstrateUnit3 * np.sum(U3S * MM)
print('\nMass AD Substrate =',round(massADSubstrateUnit3,2))



Substrate mols per element fed to Unit 3: [30.57 45.65 18.3   0.98  0.09  0.07]

AD Substrate fed to unit 3 cmol formula: [1.     1.4933 0.5987 0.0321 0.0029 0.0022]

Mols AD substrate into unit 3 = 30.567774926736302

Mass AD Substrate = 723.38


In [369]:
# Now calculate total water coming in via S2 and S3:
totalWaterS2AndS3 = (massWaterInHumanure + massWaterInNonEdibleFoodPerDay) / 1000  #divide by 1000 to get L
requiredADSubstrateConcentration = 130 #g/L
requiredADTotalWater = massADSubstrateUnit3 / requiredADSubstrateConcentration  #L
waterAddedToADPerDayNoRecycle = requiredADTotalWater - totalWaterS2AndS3
print(round(requiredADTotalWater,4))
print('The total water that needs to be added to the AD per day before considering the recycle is :', round(waterAddedToADPerDayNoRecycle,3))

5.5645
The total water that needs to be added to the AD per day before considering the recycle is : 4.352


In [370]:
#Key parameters

thetamax3=0.55 # Unit 3
mumax3=0.07 #1/day
Ks_mu3=0.04 #cmol/L
Ks_theta3=0.008/1e6  #cmol/L
Na_conc=300/1000/23 #mol/L (300mg/L in clean water)

In [371]:
#                        S       X        CH4   CO2   NH3   H2O   Na    P     R
unit3MatrixA=np.matrix([[U3S[0], X3[0],    1,    1,    0,   0,    0,    0,    S5[0]],    #C
                        [U3S[1], X3[1],    4,    0,    3,   2,    0,    0,    S5[1]],    #H
                        [U3S[2], X3[2],    0,    2,    0,   1,    0,    0,    S5[2]],    #O
                        [U3S[3], X3[3],    0,    0,    1,   0,    0,    0,    S5[3]],    #N
                        [U3S[4], X3[4],    0,    0,    0,   0,    0,    1,    S5[4]],    #P
                        [U3S[5], X3[5],    0,    0,    0,   0,    1,    0,    S5[5]],    #Na
                        [-0.35,  -1.8,     0,    0,    0,   0,    0,    0,    0    ],    #ATP
                        [0,      1,        0,    0,    0,   0,    0,    0,    0    ],    #mu
                        [0.18,   0,        0,    0,    0,   0,    0,    0,    1    ]])   #S to R ratio

In [372]:
#                   0                                  1 2 3 4 5        6                                                   7 8
Cf3 = [requiredADSubstrateConcentration/np.sum(U3S*MM),0,0,0,0,0,Na_conc*waterAddedToADPerDayNoRecycle/requiredADTotalWater,0,0]
Q3 = requiredADTotalWater
D3 = 0.67*mumax3
V3 = Q3 / D3

def response_fun_unit3(C):
    
    Cs, Cx = C[0],C[1]
    
    mu= mumax3*Cs/(Ks_mu3+Cs)
    theta=thetamax3*Cs/(Ks_theta3+Cs)

    C1_unit3 = np.matrix([[0,0,0,0,0,0,theta,mu,0]]).T  #Growth rate specified

    r1_unit3 = np.linalg.solve(unit3MatrixA,C1_unit3)
    
    return [r1_unit3[0,0],   # rs
            r1_unit3[1,0],   # rx
            r1_unit3[2,0],   # rch4
            r1_unit3[3,0],   # rco2
            r1_unit3[4,0],   # rnh3
            r1_unit3[5,0],   # rh2o
            r1_unit3[6,0],   # rna
            r1_unit3[7,0],   # rp
            r1_unit3[8,0],]  # rr

def ferm_eq_ss_unit3(C):
    
    Cs, Cx = C[0],C[1]
    
    r=response_fun_unit3([Cs, Cx])      # this is the 'fetching' from the response function
    
    dcsdt   = D3*(Cf3[0]-C[0]) + r[0]*Cx
    dcxdt   = D3*(Cf3[1]-C[1]) + r[1]*Cx
    dcnh3dt = D3*(Cf3[4]-C[2]) + r[4]*Cx
    dcnadt  = D3*(Cf3[6]-C[3]) + r[6]*Cx
    dcpdt   = D3*(Cf3[7]-C[4]) + r[7]*Cx
    dcrdt   = D3*(Cf3[8]-C[5]) + r[8]*Cx

    return [
        dcsdt,  
        dcxdt,  
        dcnh3dt,
        dcnadt, 
        dcpdt,  
        dcrdt,  
    ]

guess3=[1.5/np.sum(U3S*MM), 0.1, 0.145, 0.221, 0.0108, 0.9]
steadystate3=fsolve(ferm_eq_ss_unit3,guess3)
print(steadystate3)
print(steadystate3[0] * np.sum(U3S*MM))
print(response_fun_unit3([steadystate3[0], steadystate3[1]]))

[0.08121212 0.14003443 0.14320693 0.02111451 0.01031441 0.97418909]
1.9218746568557457
[-1.8126284166311515, 0.046900000000001066, 0.7559281144555373, 0.6835271871820057, 0.047962526212489476, -0.5499007130123804, 0.0036551162682618727, 0.003454477093760083, 0.3262731149936072]


Unit 4

In [373]:
mumax4=0.17 #1/day
thetamax4=0.57 #molATP/cmolX/day in Unit4
kla=20 #1/day
Ks_mu4=0.002 #cmol/L
Co_sat=7/1000/32 #mol/l
Ko_mu4=Co_sat*0.15 #mol/L
Ks_theta4=Ks_mu4/1e6 #cmol/L
Ko_theta4=Ko_mu4/1e6 #mol/L
Q1=1.5  #L



In [374]:
substrate4MolsPerElement = steadystate3[0] * U3S + steadystate3[1] * X3 
print('Substrate mols per element fed to Unit 4:',np.round(substrate4MolsPerElement,3))

U4S = substrate4MolsPerElement / substrate4MolsPerElement[0]      #This is the cmol formula for the AD substrate
print('\nAerobic Digester Substrate fed to unit 4 cmol formula:',np.round(U4S,4))

molsADSubstrateUnit4 = steadystate3[0] + steadystate3[1]
print('\nMols Aerobic Digester substrate into unit 4 =',molsADSubstrateUnit4)

massADSubstrateUnit4 = molsADSubstrateUnit4 * np.sum(U3S * MM)
print('\nMass Aerobic Digester Substrate =',round(massADSubstrateUnit4,2))

Substrate mols per element fed to Unit 4: [0.221 0.373 0.119 0.014 0.003 0.   ]

Aerobic Digester Substrate fed to unit 4 cmol formula: [1.0000e+00 1.6874e+00 5.3620e-01 6.2400e-02 1.3700e-02 8.0000e-04]

Mols Aerobic Digester substrate into unit 4 = 0.22124654704998215

Mass Aerobic Digester Substrate = 5.24


In [375]:
#                        S       O2,    X       CO2   NH3  H2O   Na    P    
unit4MatrixA=np.matrix([[U4S[0], 0,    X4[0],   1,    0,   0,    0,    0    ],    #C
                        [U4S[1], 0,    X4[1],   0,    3,   2,    0,    0    ],    #H
                        [U4S[2], 2,    X4[2],   2,    0,   1,    0,    0    ],    #O
                        [U4S[3], 0,    X4[3],   0,    1,   0,    0,    0    ],    #N
                        [U4S[4], 0,    X4[4],   0,    0,   0,    0,    1    ],    #P
                        [U4S[5], 0,    X4[5],   0,    0,   0,    1,    0    ],    #Na
                        [0,      -3,   -2.5,    0,    0,   0,    0,    0    ],    #ATP
                        [0,      0,    1,       0,    0,   0,    0,    0    ]])   #mu

In [376]:
#           S               O     X  Co2 Nh3 H20  Na  P
Cf4 = [steadystate3[1], Co_sat/2, 0, 0,  steadystate3[2],  0,  steadystate3[3],  steadystate3[4]]

D4 = 0.67*mumax4

def response_fun_unit4(C):
    
    Cs, Co = C[0],C[1]
    
    mu4 = mumax4*Cs/(Ks_mu4+Cs)*(Co/(Ko_mu4+Co))
    theta4 = thetamax4*Cs/(Ks_theta4+Cs)*(Co/(Ko_theta4+Co))

    C1_unit4 = np.matrix([[0,0,0,0,0,0,theta4,mu4]]).T

    r1 = np.linalg.solve(unit4MatrixA,C1_unit4)

    return [r1[0,0],   # rs
            r1[1,0],   # ro2
            r1[2,0],   # rx
            r1[3,0],   # rco2
            r1[4,0],   # rh2o
            r1[5,0],   # rnh3
            r1[6,0],   # rp
            r1[7,0]]   # rna

def ferm_eq_ss_unit4(C):
    
    Cs, Co, Cx = C[0],C[1],C[2]
    
    r=response_fun_unit4([Cs, Co])   
    
    dcsdt  =r[0]*Cx + D4*(Cf4[0]-C[0])
    dcodt  =r[1]*Cx + kla*(Co_sat-Co)
    dcxdt  =r[2]*Cx + D4*(Cf4[2]-C[2])
    dcnh3dt=r[5]*Cx + D4*(Cf4[5]-C[3])
    dcpdt  =r[6]*Cx + D4*(Cf4[6]-C[4])
    dcnadt =r[7]*Cx + D4*(Cf4[7]-C[5])
    
    return [dcsdt,  
            dcodt,  
            dcxdt,  
            dcnh3dt,
            dcpdt,
            dcnadt]

guess4=[0.4/np.sum(U3S*MM), 0.001, 0.0675, 2.6/17, 0.407/31, 0.49/23]
steadystate4=fsolve(ferm_eq_ss_unit4,guess4)
print(steadystate4)
print('\nCnh3 = {}, Cp = {}, Cna = {}'.format(steadystate4[3]*17, steadystate4[4]*31, steadystate4[5]*23))

error = guessS4 - steadystate4[3]
print('\nDifference between X4 guess and X4 calc =',error)


[ 0.00317347 -0.0003885   0.04262661  0.0747357   0.02122648  0.01134096]

Cnh3 = 1.2705069567921696, Cp = 0.658020836532799, Cna = 0.2608420542676527

Difference between X4 guess and X4 calc = -0.0002758033407158633


In [377]:
err = 1.0  # Initial error value
firstGuessS4 = 0.0  # Initial value of S4

while err >= 10e-12:
    # Calculate S4 using another method
    calculated_S4 = calculate_S4()  # Replace with your own calculation method

    err = abs(S4 - calculated_S4)  # Calculate the error

    S4 = calculated_S4  # Update S4

# The loop will exit once the error is less than 10e-12


NameError: name 'calculate_S4' is not defined